In [2]:
from backtest_python import Backtest

In [3]:
test = Backtest(1000)
test.load_data()

  100.0% [████████████████████████████████████████████████████████████████████████████████████████████████████] [+]   
 Aggregating 1 dataframe for a total of 500 candles. This might take some time. 
 Verifying data integrity. 
 Saving BTC/USDT 30m data to the database. 


In [6]:
test.data["mm20"] = test.data.close.rolling(20).mean()
test.data["achat"] = test.trier_signal(test.data.mm20 < test.data.close)
test.data["vente"] = test.trier_signal(test.data.mm20 > test.data.close)

In [7]:
for i in range(len(test.data.close)):
    if test.data.achat[i] == True:
        test.open_long(test.data.close[i], i, 10)
        test.close_short(test.data.close, i)
    elif test.data.vente[i] == True:
        test.close_long(test.data.close, i)
        test.open_short(test.data.close[i], i, 10)
    
    test.update(i, test.data.close[i])

AttributeError: 'Backtest' object has no attribute 'va_take_profit_long'